In [1]:
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pd.set_option("display.max_rows", 50)

In [2]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [9]:
#df.info()
df.var()

C:\Users\admin\AppData\Local\Temp\ipykernel_14860\1661053529.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.var()


mp               40.792717
fg               27.704656
fga              47.547526
fg%               0.003021
3p               15.174540
                   ...    
drtg_max_opp    125.635511
total_opp       160.391161
home_opp          0.250048
season            0.320218
won               0.250048
Length: 138, dtype: float64

In [670]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240,37,99,0.374,10,33,0.303,15,23,0.652,...,43.6,40.4,300,98,BRK,125,1,2021,12/22/2020,False
1,240,42,92,0.457,15,35,0.429,26,32,0.813,...,50.0,32.1,267,120,GSW,99,0,2021,12/22/2020,True
2,240,44,93,0.473,14,40,0.350,14,19,0.737,...,100.0,35.9,166,118,LAL,109,1,2021,12/22/2020,True
3,240,38,81,0.469,9,29,0.310,24,31,0.774,...,20.9,40.2,154,114,LAC,116,0,2021,12/22/2020,False
4,240,46,90,0.511,14,35,0.400,15,18,0.833,...,33.3,39.2,203,126,BOS,122,1,2021,12/23/2020,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,...,33.3,28.1,203,106,PHI,96,0,2023,4/22/2023,False
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,...,29.1,35.8,140,128,LAC,100,1,2023,4/22/2023,True
5187,240,40,92,0.435,12,37,0.324,8,10,0.800,...,25.8,30.3,152,117,PHO,112,0,2023,4/22/2023,False
5188,240,38,85,0.447,15,39,0.385,8,12,0.667,...,37.5,37.3,236,113,MIA,121,1,2023,4/22/2023,False


In [625]:
# df = df.sort_values("date")
# # df = df.reset_index(drop=True)

In [4]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240,37,99,0.374,10,33,0.303,15,23,0.652,...,43.6,40.4,300,98,BRK,125,1,2021,12/22/2020,False
1,240,42,92,0.457,15,35,0.429,26,32,0.813,...,50.0,32.1,267,120,GSW,99,0,2021,12/22/2020,True
2,240,44,93,0.473,14,40,0.350,14,19,0.737,...,100.0,35.9,166,118,LAL,109,1,2021,12/22/2020,True
3,240,38,81,0.469,9,29,0.310,24,31,0.774,...,20.9,40.2,154,114,LAC,116,0,2021,12/22/2020,False
4,240,46,90,0.511,14,35,0.400,15,18,0.833,...,33.3,39.2,203,126,BOS,122,1,2021,12/23/2020,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,...,33.3,28.1,203,106,PHI,96,0,2023,4/22/2023,False
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,...,29.1,35.8,140,128,LAC,100,1,2023,4/22/2023,True
5187,240,40,92,0.435,12,37,0.324,8,10,0.800,...,25.8,30.3,152,117,PHO,112,0,2023,4/22/2023,False
5188,240,38,85,0.447,15,39,0.385,8,12,0.667,...,37.5,37.3,236,113,MIA,121,1,2023,4/22/2023,False


In [5]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group



df = df.groupby("team", group_keys=False).apply(add_target)

In [6]:
df.tail(50)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
5140,240,38,98,0.388,5,29,0.172,18,22,0.818,...,35.1,148,105,BOS,112,1,2023,4/15/2023,False,False
5141,240,42,88,0.477,13,33,0.394,15,18,0.833,...,33.2,145,124,ATL,99,0,2023,4/15/2023,True,True
5142,240,37,88,0.420,8,29,0.276,19,22,0.864,...,35.9,157,121,CLE,97,1,2023,4/15/2023,True,False
5143,240,36,83,0.434,10,31,0.323,15,21,0.714,...,39.3,191,113,NYK,101,0,2023,4/15/2023,False,True
5144,240,39,70,0.557,13,29,0.448,10,15,0.667,...,32.0,250,124,PHI,121,1,2023,4/15/2023,False,False
5145,240,42,89,0.472,21,43,0.488,16,16,1.000,...,42.6,233,146,BRK,101,0,2023,4/15/2023,True,True
5146,240,43,90,0.478,16,50,0.320,21,27,0.778,...,33.5,206,125,SAC,126,1,2023,4/15/2023,False,False
5147,240,44,98,0.449,12,32,0.375,26,32,0.813,...,39.3,192,127,GSW,123,0,2023,4/15/2023,True,True
5148,240,30,81,0.370,11,36,0.306,9,16,0.563,...,37.6,180,95,DEN,109,1,2023,4/16/2023,False,False
5149,240,41,90,0.456,16,39,0.410,11,17,0.647,...,53.3,160,127,MIN,80,0,2023,4/16/2023,True,True


In [7]:
df[df["team"] == "PHI"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
21,240,41,87,0.471,8,28,0.286,23,30,0.767,...,36.6,207,115,WAS,107,0,2021,12/23/2020,True,True
46,240,41,88,0.466,11,31,0.355,16,22,0.727,...,35.9,139,125,NYK,89,1,2021,12/26/2020,True,False
62,240,33,79,0.418,12,37,0.324,16,22,0.727,...,34.0,200,106,CLE,118,1,2021,12/27/2020,False,True
101,240,32,84,0.381,8,31,0.258,28,33,0.848,...,30.0,152,108,TOR,93,0,2021,12/29/2020,True,True
126,240,44,91,0.484,15,33,0.455,13,15,0.867,...,30.5,131,117,ORL,92,1,2021,12/31/2020,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,240,53,101,0.525,14,32,0.438,14,19,0.737,...,36.2,123,131,BRK,105,1,2023,4/9/2023,True,True
5145,240,42,89,0.472,21,43,0.488,16,16,1.000,...,42.6,233,146,BRK,101,0,2023,4/15/2023,True,True
5157,240,36,80,0.450,11,35,0.314,13,16,0.813,...,26.6,119,117,BRK,84,0,2023,4/17/2023,True,True
5172,240,40,82,0.488,13,31,0.419,9,13,0.692,...,42.4,229,125,BRK,97,1,2023,4/20/2023,True,True


In [8]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

C:\Users\huyda\AppData\Local\Temp\ipykernel_3500\2793819745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [9]:
df["won"].value_counts()

False    2595
True     2595
Name: won, dtype: int64

In [10]:
df["target"].value_counts()

0    2580
1    2580
2      30
Name: target, dtype: int64

In [11]:
nulls = pd.isnull(df).sum()

In [12]:
nulls = nulls[nulls > 0]

In [13]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [14]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [15]:
df = df[valid_columns].copy()

In [16]:
df


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240,37,99,0.374,10,33,0.303,15,23,0.652,...,40.4,300,98,BRK,125,1,2021,12/22/2020,False,0
1,240,42,92,0.457,15,35,0.429,26,32,0.813,...,32.1,267,120,GSW,99,0,2021,12/22/2020,True,1
2,240,44,93,0.473,14,40,0.350,14,19,0.737,...,35.9,166,118,LAL,109,1,2021,12/22/2020,True,1
3,240,38,81,0.469,9,29,0.310,24,31,0.774,...,40.2,154,114,LAC,116,0,2021,12/22/2020,False,1
4,240,46,90,0.511,14,35,0.400,15,18,0.833,...,39.2,203,126,BOS,122,1,2021,12/23/2020,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,...,28.1,203,106,PHI,96,0,2023,4/22/2023,False,2
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,...,35.8,140,128,LAC,100,1,2023,4/22/2023,True,2
5187,240,40,92,0.435,12,37,0.324,8,10,0.800,...,30.3,152,117,PHO,112,0,2023,4/22/2023,False,2
5188,240,38,85,0.447,15,39,0.385,8,12,0.667,...,37.3,236,113,MIA,121,1,2023,4/22/2023,False,2


In [17]:
rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

In [18]:

removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [19]:
scaler = MinMaxScaler()
df.loc[:, selected_columns] = scaler.fit_transform(df[selected_columns])

C:\Users\huyda\AppData\Local\Temp\ipykernel_3500\4041474412.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, selected_columns] = scaler.fit_transform(df[selected_columns])


In [20]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.350,0.611111,0.236585,0.296296,0.433962,0.372323,0.307692,0.422222,0.497110,...,0.234917,1.000000,0.168831,BRK,0.611765,1.0,2021,12/22/2020,False,0
1,0.0,0.475,0.481481,0.439024,0.481481,0.471698,0.579901,0.589744,0.622222,0.729769,...,0.128370,0.829897,0.454545,GSW,0.305882,0.0,2021,12/22/2020,True,1
2,0.0,0.525,0.500000,0.478049,0.444444,0.566038,0.449753,0.282051,0.333333,0.619942,...,0.177150,0.309278,0.428571,LAL,0.423529,1.0,2021,12/22/2020,True,1
3,0.0,0.375,0.277778,0.468293,0.259259,0.358491,0.383855,0.538462,0.600000,0.673410,...,0.232349,0.247423,0.376623,LAC,0.505882,0.0,2021,12/22/2020,False,1
4,0.0,0.575,0.444444,0.570732,0.444444,0.471698,0.532125,0.307692,0.311111,0.758671,...,0.219512,0.500000,0.532468,BOS,0.576471,1.0,2021,12/23/2020,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,0.0,0.250,0.259259,0.331707,0.259259,0.509434,0.273476,0.256410,0.333333,0.543353,...,0.077022,0.500000,0.272727,PHI,0.270588,0.0,2023,4/22/2023,False,2
5186,0.0,0.450,0.333333,0.514634,0.259259,0.226415,0.546952,0.461538,0.511111,0.679191,...,0.175866,0.175258,0.558442,LAC,0.317647,1.0,2023,4/22/2023,True,2
5187,0.0,0.425,0.481481,0.385366,0.370370,0.509434,0.406919,0.128205,0.133333,0.710983,...,0.105263,0.237113,0.415584,PHO,0.458824,0.0,2023,4/22/2023,False,2
5188,0.0,0.375,0.351852,0.414634,0.481481,0.547170,0.507414,0.128205,0.177778,0.518786,...,0.195122,0.670103,0.363636,MIA,0.564706,1.0,2023,4/22/2023,False,2


In [21]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [22]:
predictors = list(selected_columns[sfs.get_support()])

In [23]:
predictors

['fg%',
 '3p',
 '3pa',
 '3p%',
 'pts',
 'efg%',
 'ast%',
 'usg%',
 'ortg',
 'stl_max',
 'blk_max',
 '+/-_max',
 'ts%_max',
 'efg%_max',
 'ftr_max',
 'stl%_max',
 'ortg_max',
 'drtg_max',
 'total',
 'fg_opp',
 'tov_opp',
 'blk%_opp',
 'usg%_opp',
 'drtg_opp',
 'ft%_max_opp',
 'orb_max_opp',
 '+/-_max_opp',
 '3par_max_opp',
 'orb%_max_opp',
 'tov%_max_opp']

In [24]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [25]:
predictions = backtest(df, rr, predictors)

In [26]:
predictions

,actual,prediction
4988,0,1
4989,1,1
4990,1,0
4991,0,1
4992,1,0
...,...,...
5185,2,1
5186,2,1
5187,2,0
5188,2,0


In [27]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.4158415841584158

In [28]:

df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

home
0.0    0.452023
1.0    0.547977
dtype: float64

In [650]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.350,0.611111,0.236585,0.296296,0.433962,0.372323,0.307692,0.422222,0.497110,...,0.234917,1.000000,0.168831,BRK,0.611765,1.0,2021,12/22/2020,False,0
1,0.0,0.475,0.481481,0.439024,0.481481,0.471698,0.579901,0.589744,0.622222,0.729769,...,0.128370,0.829897,0.454545,GSW,0.305882,0.0,2021,12/22/2020,True,1
2,0.0,0.525,0.500000,0.478049,0.444444,0.566038,0.449753,0.282051,0.333333,0.619942,...,0.177150,0.309278,0.428571,LAL,0.423529,1.0,2021,12/22/2020,True,1
3,0.0,0.375,0.277778,0.468293,0.259259,0.358491,0.383855,0.538462,0.600000,0.673410,...,0.232349,0.247423,0.376623,LAC,0.505882,0.0,2021,12/22/2020,False,1
4,0.0,0.575,0.444444,0.570732,0.444444,0.471698,0.532125,0.307692,0.311111,0.758671,...,0.219512,0.500000,0.532468,BOS,0.576471,1.0,2021,12/23/2020,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,0.0,0.250,0.259259,0.331707,0.259259,0.509434,0.273476,0.256410,0.333333,0.543353,...,0.077022,0.500000,0.272727,PHI,0.270588,0.0,2023,4/22/2023,False,2
5186,0.0,0.450,0.333333,0.514634,0.259259,0.226415,0.546952,0.461538,0.511111,0.679191,...,0.175866,0.175258,0.558442,LAC,0.317647,1.0,2023,4/22/2023,True,2
5187,0.0,0.425,0.481481,0.385366,0.370370,0.509434,0.406919,0.128205,0.133333,0.710983,...,0.105263,0.237113,0.415584,PHO,0.458824,0.0,2023,4/22/2023,False,2
5188,0.0,0.375,0.351852,0.414634,0.481481,0.547170,0.507414,0.128205,0.177778,0.518786,...,0.195122,0.670103,0.363636,MIA,0.564706,1.0,2023,4/22/2023,False,2


In [31]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    rolling = team.rolling(10).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\huyda\AppData\Local\Temp\ipykernel_3500\959676254.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['team'], dtype='object')
  rolling = team.rolling(10).mean()


In [33]:
df_rolling.head(50)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.000000,0.4875,0.444444,0.480488,0.414815,0.483019,0.483031,0.366667,0.413333,0.639306,...,0.0549,0.0688,0.408491,0.281001,0.451031,0.437662,0.500000,0.5,0.5,2021.0


In [34]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [35]:
df = df.dropna()

In [39]:
df.loc[:, ['target', 'date', 'season', 'team', 'team_opp']].tail(50)

,target,date,season,team,team_opp
5140,0,4/15/2023,2023,ATL,BOS
5141,1,4/15/2023,2023,BOS,ATL
5142,0,4/15/2023,2023,NYK,CLE
5143,1,4/15/2023,2023,CLE,NYK
5144,0,4/15/2023,2023,BRK,PHI
5145,1,4/15/2023,2023,PHI,BRK
5146,0,4/15/2023,2023,GSW,SAC
5147,1,4/15/2023,2023,SAC,GSW
5148,0,4/16/2023,2023,MIN,DEN
5149,1,4/16/2023,2023,DEN,MIN


In [40]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

In [657]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,season_10,home_next,team_opp_next,date_next
243,0.0,0.450,0.370370,0.487805,0.370370,0.415094,0.490939,0.487179,0.577778,0.614162,...,0.150193,0.509278,0.474026,0.449412,0.6,0.7,2021.0,1.0,ORL,1/15/2021
246,0.0,0.425,0.314815,0.500000,0.444444,0.396226,0.617792,0.205128,0.200000,0.777457,...,0.244801,0.391753,0.481818,0.425882,0.6,0.6,2021.0,1.0,CHI,1/10/2021
250,0.0,0.575,0.462963,0.556098,0.296296,0.301887,0.507414,0.256410,0.222222,0.897399,...,0.270988,0.442784,0.448052,0.555294,0.6,0.4,2021.0,0.0,LAC,1/10/2021
251,0.0,0.450,0.277778,0.558537,0.370370,0.358491,0.555189,0.512821,0.511111,0.786127,...,0.230809,0.351546,0.487013,0.395294,0.4,0.7,2021.0,0.0,HOU,1/10/2021
252,0.0,0.375,0.277778,0.468293,0.444444,0.396226,0.617792,0.435897,0.377778,0.930636,...,0.167908,0.390722,0.468831,0.451765,0.3,0.5,2021.0,1.0,OKC,1/10/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,0.0,0.200,0.166667,0.331707,0.333333,0.415094,0.439868,0.461538,0.600000,0.533237,...,0.174711,0.389691,0.429870,0.322353,0.5,0.5,2022.0,1.0,GSW,6/16/2022
4985,0.0,0.450,0.407407,0.460976,0.259259,0.566038,0.243822,0.256410,0.244444,0.807803,...,0.321566,0.611340,0.519481,0.374118,0.4,0.7,2022.0,0.0,BOS,6/16/2022
4986,0.0,0.375,0.481481,0.331707,0.629630,0.679245,0.553542,0.128205,0.088889,1.000000,...,0.308601,0.596392,0.523377,0.377647,0.5,0.7,2022.0,NaN,None,None
4987,0.0,0.275,0.259259,0.360976,0.333333,0.339623,0.520593,0.205128,0.177778,0.880058,...,0.172144,0.412887,0.418182,0.315294,0.5,0.5,2022.0,NaN,None,None


In [56]:
df.loc[:, ['won','target', 'team', 'date', 'team_opp', 'team_opp_next', 'date_next', 'home_next']].tail(20)

,won,target,team,date,team_opp,team_opp_next,date_next,home_next
5170,False,1,MIA,4/19/2023,MIL,MIL,4/22/2023,1.0
5171,True,0,MIL,4/19/2023,MIA,MIA,4/22/2023,0.0
5172,True,1,PHI,4/20/2023,BRK,BRK,4/22/2023,0.0
5173,False,0,BRK,4/20/2023,PHI,PHI,4/22/2023,1.0
5174,False,2,SAC,4/20/2023,GSW,None,None,NaN
5175,True,2,GSW,4/20/2023,SAC,None,None,NaN
5176,True,1,PHO,4/20/2023,LAC,LAC,4/22/2023,0.0
5177,False,0,LAC,4/20/2023,PHO,PHO,4/22/2023,1.0
5178,False,2,BOS,4/21/2023,ATL,None,None,NaN
5179,True,2,ATL,4/21/2023,BOS,None,None,NaN


In [44]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])


In [49]:
full.loc[:, ['won','target', 'team_x', 'date', 'team_opp', 'team_opp_next_x', 'date_next']].tail(50)


,won,target,team_x,date,team_opp,team_opp_next_y,date_next
5076,False,1,NYK,4/9/2023,IND,NYK,4/15/2023
5077,False,0,MEM,4/9/2023,OKC,MEM,4/16/2023
5078,True,1,OKC,4/9/2023,MEM,OKC,4/12/2023
5079,True,1,LAC,4/9/2023,PHO,LAC,4/16/2023
5080,False,0,PHO,4/9/2023,LAC,PHO,4/16/2023
5081,True,0,GSW,4/9/2023,POR,GSW,4/15/2023
5082,False,0,MIL,4/9/2023,TOR,MIL,4/16/2023
5083,True,0,TOR,4/9/2023,MIL,TOR,4/12/2023
5084,False,1,MIN,4/11/2023,LAL,MIN,4/14/2023
5085,True,1,LAL,4/11/2023,MIN,LAL,4/16/2023


In [50]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]].tail(50)

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
5076,NYK,CLE,CLE,NYK,4/15/2023
5077,MEM,LAL,LAL,MEM,4/16/2023
5078,OKC,NOP,NOP,OKC,4/12/2023
5079,LAC,PHO,PHO,LAC,4/16/2023
5080,PHO,LAC,LAC,PHO,4/16/2023
5081,GSW,SAC,SAC,GSW,4/15/2023
5082,MIL,MIA,MIA,MIL,4/16/2023
5083,TOR,CHI,CHI,TOR,4/12/2023
5084,MIN,OKC,OKC,MIN,4/14/2023
5085,LAL,MEM,MEM,LAL,4/16/2023


In [51]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [52]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_opp_next_x',
 'date_next',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [53]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [57]:
predictors = list(selected_columns[sfs.get_support()])

In [58]:
predictors

['usg%',
 '+/-_max',
 'ortg_max',
 'ft%_opp',
 '3par_opp',
 'usg%_opp',
 '3p_max_opp',
 'blk_max_opp',
 'stl%_max_opp',
 'usg%_10_x',
 '3p%_max_10_x',
 'stl%_max_10_x',
 'home_10_x',
 'usg%_opp_10_x',
 'home_opp_10_x',
 'home_next',
 'fta_10_y',
 'stl_10_y',
 'stl%_10_y',
 'usg%_10_y',
 'ast%_max_10_y',
 'stl%_max_10_y',
 'blk%_max_10_y',
 'home_10_y',
 'stl_opp_10_y',
 'pts_opp_10_y',
 'usg%_opp_10_y',
 'drtg_max_opp_10_y',
 'total_opp_10_y',
 'home_opp_10_y']

In [59]:
predictions = backtest(full, rr, predictors)

In [61]:
predictions.tail(50)

,actual,prediction
5076,1,0
5077,0,0
5078,1,0
5079,1,0
5080,0,1
5081,0,1
5082,0,1
5083,0,1
5084,1,1
5085,1,0


In [60]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5398773006134969